In [ ]:
#EXPORT MODEL TO ONNX
import torch
from torchvision import models

# === Load pretrained ResNet-50 ===
model = models.resnet50(pretrained=True)
model.eval()

# === Example input tensor ===
dummy_input = torch.randn(1, 3, 224, 224)

# === Export to ONNX ===
onnx_path = "resnet50_fp32.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
    opset_version=12
)

print(f"✅ Exported model to: {onnx_path}")


In [ ]:
pip install onnx onnxruntime onnxruntime-tools

In [ ]:
#STATIC QUANTIZATION
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType
import onnx
from PIL import Image
from torchvision import transforms
import numpy as np
from pathlib import Path

# === Dataset for calibration ===
class ImagenetCalibrationReader(CalibrationDataReader):
    def __init__(self, image_dir, num_images=100):
        self.image_paths = list(Path(image_dir).glob("*.JPEG"))[:num_images]
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        self.enum_data = None

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter(self._preprocess())
        return next(self.enum_data, None)

    def _preprocess(self):
        for img_path in self.image_paths:
            img = Image.open(img_path).convert("RGB")
            tensor = self.transform(img).unsqueeze(0).numpy()
            yield {"input": tensor.astype(np.float32)}

# === Run quantization ===
fp32_model_path = "resnet50_fp32.onnx"
int8_model_path = "resnet50_int8.onnx"
calibration_reader = ImagenetCalibrationReader("image_classification/mlperf_inference/dataset/imagenet/val_flat", num_images=100)

quantize_static(
    model_input=fp32_model_path,
    model_output=int8_model_path,
    calibration_data_reader=calibration_reader,
    quant_format=QuantType.QOperator
)

print(f"✅ Quantized model saved to: {int8_model_path}")

In [ ]:
#INFERENCE TO CHECK IF QUANTIZED MODEL WORKS WITH CPU
import onnxruntime as ort
import numpy as np
from PIL import Image
from torchvision import transforms
from pathlib import Path

# === Paths ===
model_path = "resnet50_int8.onnx"
image_path = Path("mlperf_inference/datasets/imagenet/val_flat/ILSVRC2012_val_00000001.JPEG")

# === Preprocess image ===
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
img = Image.open(image_path).convert("RGB")
input_tensor = transform(img).unsqueeze(0).numpy()

# === Run inference ===
session = ort.InferenceSession(model_path)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
output = session.run([output_name], {input_name: input_tensor})[0]

# === Show top-1 prediction
predicted_idx = np.argmax(output)
print(f"✅ ONNX Inference successful! Top-1 class index: {predicted_idx}")
